<a href="https://colab.research.google.com/github/igt-cb/sfdx-scanner/blob/main/copy_of_sds_phd19_nlp_longtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here a nice [cheat-sheet
](https://github.com/abhat222/Data-Science--Cheat-Sheet/raw/master/Python/spaCy.pdf)

In [ ]:
# @title Interactive Topic Plot { vertical-output: true }
# -*- coding: utf-8 -*-
!pip install -qq gutenberg-cleaner holoviews bokeh==2.4.0 datashader > /dev/null
import itertools
import numpy as np
from collections import Counter
import nltk #this part is needed on colab.
nltk.download('punkt')
nltk.download('stopwords')
#----------------------------------------
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
article = article_text
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 5000000
# Let's apply the model to the article (as easy as that)
article_nlp = nlp(article)
# Yes spacy recognises entities in the text out of the box (among many other things)
[(ent.text, ent.label_) for ent in article_nlp.ents][:20]
import requests as rq
ents_from_sents = []
ents_from_sents=article_nlp.ents
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
for i in ents_from_sents:
  G.add_edges_from(list(itertools.combinations(i,2)))
degree_centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, degree_centrality, 'degree')
perc_filter = np.percentile([v for u,v in degree_centrality.items()], 80)
nodes_selected = [x for x,y in degree_centrality.items() if y >= perc_filter]

G = G.subgraph(nodes_selected)
plt.figure(figsize=(12,12))
nx.draw_kamada_kawai(G, with_labels = True, node_size=[v * 100 for v in dict(G.degree).values()])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title Bokeh Interactive

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show
kwargs = dict(width=800, height=800, xaxis=None, yaxis=None)
opts.defaults(opts.Nodes(**kwargs), opts.Graph(**kwargs))
G_layout = nx.layout.kamada_kawai_layout(G)
g_plot = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'])
labels = hv.Labels(g_plot.nodes, ['x', 'y'], 'index')
from holoviews.operation.datashader import datashade, bundle_graph
bundled = bundle_graph(g_plot)
show(hv.render(bundled * labels.opts(text_font_size='10pt', text_color='white', bgcolor='gray')))